In [53]:
import numpy as np
import pandas as pd
from tabulate import tabulate
import plotly.express as px

In [2]:
data = pd.read_csv("AdSmartABdata.csv")


In [28]:
data.head() # Displaying the data

,auction_id,experiment,date,hour,device_make,platform_os,browser,yes,no
0,0008ef63-77a7-448b-bd1e-075f42c55e39,exposed,2020-07-10,8,Generic Smartphone,6,Chrome Mobile,0,0
1,000eabc5-17ce-4137-8efe-44734d914446,exposed,2020-07-07,10,Generic Smartphone,6,Chrome Mobile,0,0
2,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-05,2,E5823,6,Chrome Mobile WebView,0,1
3,00187412-2932-4542-a8ef-3633901c98d9,control,2020-07-03,15,Samsung SM-A705FN,6,Facebook,0,0
4,001a7785-d3fe-4e11-a344-c8735acacc2c,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0,0


In [29]:
data.info() # getting the information of our dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8077 entries, 0 to 8076
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   auction_id   8077 non-null   object
 1   experiment   8077 non-null   object
 2   date         8077 non-null   object
 3   hour         8077 non-null   int64 
 4   device_make  8077 non-null   object
 5   platform_os  8077 non-null   int64 
 6   browser      8077 non-null   object
 7   yes          8077 non-null   int64 
 8   no           8077 non-null   int64 
dtypes: int64(4), object(5)
memory usage: 568.0+ KB


In [30]:
data.isna().sum()

auction_id     0
experiment     0
date           0
hour           0
device_make    0
platform_os    0
browser        0
yes            0
no             0
dtype: int64

In [32]:
data[data["yes"] == 0]

,auction_id,experiment,date,hour,device_make,platform_os,browser,yes,no
0,0008ef63-77a7-448b-bd1e-075f42c55e39,exposed,2020-07-10,8,Generic Smartphone,6,Chrome Mobile,0,0
1,000eabc5-17ce-4137-8efe-44734d914446,exposed,2020-07-07,10,Generic Smartphone,6,Chrome Mobile,0,0
2,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-05,2,E5823,6,Chrome Mobile WebView,0,1
3,00187412-2932-4542-a8ef-3633901c98d9,control,2020-07-03,15,Samsung SM-A705FN,6,Facebook,0,0
4,001a7785-d3fe-4e11-a344-c8735acacc2c,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0,0
...,...,...,...,...,...,...,...,...,...
8072,ffea24ec-cec1-43fb-b1d1-8f93828c2be2,exposed,2020-07-05,7,Generic Smartphone,6,Chrome Mobile,0,0
8073,ffea3210-2c3e-426f-a77d-0aa72e73b20f,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0,0
8074,ffeaa0f1-1d72-4ba9-afb4-314b3b00a7c7,control,2020-07-04,9,Generic Smartphone,6,Chrome Mobile,0,0
8075,ffeeed62-3f7c-4a6e-8ba7-95d303d40969,exposed,2020-07-05,15,Samsung SM-A515F,6,Samsung Internet,0,0


## Task 1: A/B testing framework


### Task 1.1: Understanding A/B testing framework

Which online users belong to the control and exposed groups?


In [42]:
Controled = data[data['experiment']=="control"]
Controled.head()

,auction_id,experiment,date,hour,device_make,platform_os,browser,yes,no
3,00187412-2932-4542-a8ef-3633901c98d9,control,2020-07-03,15,Samsung SM-A705FN,6,Facebook,0,0
4,001a7785-d3fe-4e11-a344-c8735acacc2c,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0,0
5,0027ce48-d3c6-4935-bb12-dfb5d5627857,control,2020-07-03,15,Samsung SM-G960F,6,Facebook,0,0
6,002e308b-1a07-49d6-8560-0fbcdcd71e4b,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0,0
7,00393fb9-ca32-40c0-bfcb-1bd83f319820,control,2020-07-09,5,Samsung SM-G973F,6,Facebook,0,0


In [40]:
Controled.describe()

,hour,platform_os,yes,no
count,4071.000000,4071.000000,4071.000000,4071.000000
mean,12.499140,5.924343,0.064849,0.079096
std,5.331095,0.264481,0.246289,0.269922
min,0.000000,5.000000,0.000000,0.000000
25%,9.000000,6.000000,0.000000,0.000000
50%,15.000000,6.000000,0.000000,0.000000
75%,15.000000,6.000000,0.000000,0.000000
max,23.000000,6.000000,1.000000,1.000000


In [41]:
Exposed = data[data['experiment']=="exposed"]
Exposed.head()

,auction_id,experiment,date,hour,device_make,platform_os,browser,yes,no
0,0008ef63-77a7-448b-bd1e-075f42c55e39,exposed,2020-07-10,8,Generic Smartphone,6,Chrome Mobile,0,0
1,000eabc5-17ce-4137-8efe-44734d914446,exposed,2020-07-07,10,Generic Smartphone,6,Chrome Mobile,0,0
2,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-05,2,E5823,6,Chrome Mobile WebView,0,1
8,004940f5-c642-417a-8fd2-c8e5d989f358,exposed,2020-07-04,0,Generic Smartphone,6,Chrome Mobile WebView,0,0
13,006b9c6e-5f5d-4385-a811-ff20a24b30ac,exposed,2020-07-06,8,Samsung SM-G973F,6,Chrome Mobile WebView,0,0


In [9]:
Exposed.describe()

,hour,platform_os,yes,no
count,4006.000000,4006.000000,4006.000000,4006.000000
mean,10.716675,5.970295,0.076885,0.087119
std,5.986323,0.171259,0.266442,0.282045
min,0.000000,5.000000,0.000000,0.000000
25%,6.000000,6.000000,0.000000,0.000000
50%,10.000000,6.000000,0.000000,0.000000
75%,15.000000,6.000000,0.000000,0.000000
max,23.000000,7.000000,1.000000,1.000000


### Task 1.2

•	Perform data exploration to count unique values of categorical variables, make histogram, relational, and other necessary plots to help understand the data. For each of the plots you produce, write a description of what the plot shows in markdown cells.

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8077 entries, 0 to 8076
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   auction_id   8077 non-null   object
 1   experiment   8077 non-null   object
 2   date         8077 non-null   object
 3   hour         8077 non-null   int64 
 4   device_make  8077 non-null   object
 5   platform_os  8077 non-null   int64 
 6   browser      8077 non-null   object
 7   yes          8077 non-null   int64 
 8   no           8077 non-null   int64 
dtypes: int64(4), object(5)
memory usage: 568.0+ KB


In [45]:
data['experiment'].value_counts()

control    4071
exposed    4006
Name: experiment, dtype: int64

In [50]:
data['hour'].value_counts().head(10)

15    1895
8      394
7      381
9      346
10     336
16     335
6      327
14     319
5      302
13     290
Name: hour, dtype: int64

In [51]:
data['device_make'].value_counts().head(10)

Generic Smartphone    4743
iPhone                 433
Samsung SM-G960F       203
Samsung SM-G973F       154
Samsung SM-G950F       148
Samsung SM-G930F       100
Samsung SM-G975F        97
Samsung SM-A202F        88
Samsung SM-A405FN       87
Samsung SM-J330FN       69
Name: device_make, dtype: int64

In [15]:
data['platform_os'].value_counts()

6    7648
5     428
7       1
Name: platform_os, dtype: int64

In [16]:
data['browser'].value_counts()

Chrome Mobile                 4554
Chrome Mobile WebView         1489
Samsung Internet               824
Facebook                       764
Mobile Safari                  337
Chrome Mobile iOS               51
Mobile Safari UI/WKWebView      44
Opera Mobile                     3
Chrome                           3
Pinterest                        3
Android                          1
Opera Mini                       1
Firefox Mobile                   1
Puffin                           1
Edge Mobile                      1
Name: browser, dtype: int64

In [17]:
data['yes'].value_counts()

0    7505
1     572
Name: yes, dtype: int64

In [18]:
data['no'].value_counts()

0    7406
1     671
Name: no, dtype: int64

#### Plots

In [61]:
fig = px.histogram(data, x="experiment", title="Experiment distribution")
fig.show()

* The graph shows that the number rof control and expeosed is almost the same. around 4000 for each case

In [62]:
fig = px.histogram(data, x="date", title="date distribution")
fig.show()

* Most users visited the website on 2020-07-03

In [70]:
fig = px.histogram(data, x="hour", title="Hour distribution")
fig.update_layout(bargap=0.5)
fig.show()

* Most users visited the website around 15:00

In [72]:
fig = px.histogram(data, x="device_make", title="device_make distribution")
fig.update_layout(bargap=0.2)
fig.show()

* Most respondents use Generic Smartphone

In [69]:
fig = px.histogram(data, x="platform_os", title="platform_os distribution")
fig.update_layout(bargap=0.5)
fig.show()

* The platform os of 6 was mot most used

In [74]:
fig = px.histogram(data, x="browser", title="browser distribution")
fig.update_layout(bargap=0.2)
fig.show()

Chrome Mobile is the most used browser, with 4554 users

In [78]:
fig = px.histogram(data, x="yes", title="Those who know the brand Lux")
fig.update_layout(bargap=0.5)
fig.show()

In [77]:
fig = px.histogram(data, x="no", title="Those who do not know the brand Lux")
fig.update_layout(bargap=0.5)
fig.show()